# __텍스트로 텍스트 검색하기__

- 튜토리얼 난이도: ★★☆☆☆
- 읽는데 걸리는 시간: 7분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_search/search_text_by_text.ipynb
- 참고 문서: [Naver sentiment movie corpus v1.0](https://github.com/e9t/nsmc#naver-sentiment-movie-corpus-v10), [단어 임베딩: 어휘의 의미(LEXICAL SEMANTICS)를 인코딩하기](https://tutorials.pytorch.kr/beginner/nlp/word_embeddings_tutorial.html), [컴퓨터가 바라보는 문자](https://heung-bae-lee.github.io/2020/01/16/NLP_01/)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">텍스트 수치화 기술 이해하기</h4>
    <p>컴퓨터는 사람이 쓰는 언어(자연어)를 곧바로 입력으로 받을 수 없습니다. 때문에 자연어를 기계가 인식할 수 있는 수치 데이터로 변환하는 과정이 필요합니다. 자연어 처리분야에서 임베딩이란 이렇게 사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자형태인 vector로 바꾼 결과 혹은 그 일련의 과정 전체를 의미합니다.</p>
</div>

자연어의 임베딩을 구하는 기법은 크게 통계적인 기법과, 인공신경망 기반 기법으로 구분되고, 세부적으로도 더 나눌 수 있습니다. ThanoSQL에서는 인공신경망 기반의 [자가학습모델(Self-Supervised Learning Model)](https://ko.wikipedia.org/wiki/%EC%9E%90%EA%B8%B0_%EC%A7%80%EB%8F%84_%ED%95%99%EC%8A%B5)을 사용하여, 텍스트를 잘 표현하는 벡터를 스스로 학습하는 방법을 제공합니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 이번 튜토리얼에서는 영화 리뷰 데이터를 사용합니다. 데이터는 영화 리뷰 텍스트와, 라벨값으로 구성되어있지만, 자가학습법을 사용해 훈련하는 예제를 보여드리기 위해 라벨값은 사용하지 않습니다. 10000개의 영화 리뷰 데이터를 학습하여, 입력 문장과 유사한 문장 찾기, 문장의 키워드 추출하기를 해보겠습니다.</p>
</div>

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/how_to_use_ThanoSQL/#5-thanosql)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET nsmc_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY nsmc_train
OPTIONS (overwrite=True)
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_train.csv'

Success


In [4]:
%%thanosql
COPY nsmc_test
OPTIONS (overwrite=True)
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 데이터 세트명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 데이터베이스 상에 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

텍스트 수치화 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __nsmc_train__ 테이블을 사용합니다. nsmc_train 테이블은 NAVER Sentiment Movie Corpus 영화 리뷰 데이터 및 라벨 정보의 일부가 담겨 있는 테이블입니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM nsmc_train
LIMIT 5

,id,document,label
0,9465891,저퀄리티. 뭐하나 장점이 없음.,0
1,9792014,10점 만점에 1점 !!! 최악이네 하아...,0
2,3795348,누가 억지눈물 잘흘리나 자기네들끼리 대결하는 작품,0
3,1129393,신도 싫고 인간도 싫다,1
4,4678610,추억의 명작만화,1


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><storng>nsmc_train</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>id: 데이터의 식별 번호</li>
        <li>document: 영화 리뷰 데이터</li>
        <li>label: 라벨값</li>
    </ul>
</div>

## __2. 텍스트 수치화 모델 생성__

이전 단계에서 확인한 __nsmc_train__ 테이블을 사용하여 텍스트 수치화 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>nsmc_text_search_model</strong>이라는 이름의 모델을 만듭니다.\
(쿼리 실행 시 예상 소요 시간: 2 min)

In [6]:
%%thanosql
BUILD MODEL nsmc_text_search_model
USING SBERTKo
OPTIONS (
    text_col="document",
    overwrite=True
)
AS
SELECT *
FROM nsmc_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>nsmc_text_search_model</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>SBERTKo</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"max_epochs": 텍스트 수치화 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, optional, default: 1)</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

다음 "__CONVERT USING__ " 쿼리 구문을 실행하여 nsmc_test 텍스트들을 수치화 합니다. 수치화 된 결과는 __nsmc_test__ 테이블에 사용자가 옵션으로 지정한 이름(default: convert_result)의 컬럼에 저장됩니다.

In [7]:
%%thanosql
CONVERT USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    table_name='nsmc_test',
    batch_size=32,
    result_col='convert_result'
    )
AS
SELECT *
FROM nsmc_test

,id,document,label,convert_result
0,9181084,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[0.024359047, -0.022454703, -0.018680774, -0.0..."
1,6318649,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[0.00087407755, -0.0067267334, 0.025690123, -0..."
2,5718332,유쾌하고 신나는 스피드 코미디.,1,"[-0.011161274, -0.02666556, 0.045139965, 0.008..."
3,8063675,정말 따뜻했던 드라마ㅠㅠ,1,"[0.050167948, -0.0060978937, 0.002397817, -0.0..."
4,10229366,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[0.008381872, -0.009418637, 0.094107054, 0.001..."
...,...,...,...,...
1995,5382227,30년간 본 수천편의 드라마중 최고는 이 작품,1,"[0.023315229, -0.013492188, 0.021891769, -0.02..."
1996,4584773,빛돌을 이길 5명의 용사들의 패배,1,"[-0.043673623, 0.013066697, 0.024431909, 0.051..."
1997,2559484,빌머레이의 굳은 표정에 모든것이 살아있다.. 짐자무시 최고!,1,"[0.01111976, 0.026474133, -0.037597883, 0.0530..."
1998,9867081,이런영화에 투자하지 않는것이 진정한 한국영화의 발전을위한 초석,0,"[0.031167667, 0.041263472, 0.052478615, 0.0274..."


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" 쿼리 구문은 <strong>nsmc_text_search_model</strong> 모델을 텍스트 수치화를 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"table_name": ThanoSQL 워크스페이스 데이터베이스 내에 저장될 테이블 이름 (str, optional)</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"result_col": 데이터 테이블에서 수치화된 결과를 담을 컬럼 이름 (str, optional, default: 'convert_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 텍스트 수치화 모델을 사용하여 유사한 문서 검색__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서를 검색합니다.

In [8]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='영화 내용이 불만족스러웠다',
        emb_col='convert_result',
        result_col='score'
        )
    AS 
    SELECT *
    FROM nsmc_test
    )
ORDER BY score DESC
LIMIT 10

,document,label,score
0,너무 작위적인 스토리 전개..실망스러운 영화ㅠ,0,0.662901
1,내가 생각했던 스토리와 조금 달랐지만 신선하고 독특했던 영화였다.,1,0.660392
2,영화 전개가 너무 지루해요.,0,0.646023
3,소리가 너무 안들려서 대사전달이 않됐던 영화.,0,0.636529
4,시나리오는 한없이 후진데다가 썩은 동선이 가득한 영화,0,0.636151
5,상황설정은 좋았으나 너무 유치하게 이끌어간다,0,0.635902
6,감동적이긴 하지만 동성애를 정당화 시키려는 영화라 좀 그렇네요,0,0.631703
7,"평점보고 기대해서 그런지 실망스런 영화 ,쏘쏘한 영화",0,0.631166
8,모든것이 흥미로운 영화였다.,1,0.628783
9,왕건 후속편으로써 매우 실망스런 작품.,0,0.626032


In [10]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='기분이 좋아지는 작품',
        emb_col='convert_result',
        result_col='score'
        )
    AS
    SELECT *
    FROM nsmc_test
    )
ORDER BY score DESC
LIMIT 10

,document,label,score
0,보고나면 기분 좋은 영화,1,0.722369
1,너무 멋있는 영화. 환한 느낌을 준다,1,0.718308
2,몰입도 잘되고 좋은 영화,1,0.692874
3,보는 내내 잔잔한 미소가 지어지는 영화. 거기다 음악도 좋고,1,0.688710
4,심영이후 감동적인 작품,1,0.672638
5,뻔한 스토리지만 과거 추억을 생각나게 해주는 영화 재미있었어요,1,0.663348
6,감동적인 영화,1,0.651054
7,재미있었어요,1,0.650601
8,이제서야 봤는데 독특하면서도 너무 아름다운 영화여서 기억에 남을거같아요.,1,0.638538
9,"내겐 강렬했고, 많은 생각이 들게했고, 오묘한 기분을 느끼게 한 영화",1,0.636940


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 검색 시 필요한 변수들을 정의합니다.
        <ul>
                <li>"search_by": 검색할 때 사용할 이미지|텍스트|오디오|비디오 타입 (str)</li>
                <li>"search_input": 검색할 때 사용할 입력값 (str)</li>
                <li>"emb_col": 데이터 테이블에서 수치화된 결과를 담은 컬럼 (str)</li>
                <li>"result_col": 데이터 테이블에서 검색 결과를 담을 컬럼 이름 (str, optional, default: 'search_result')</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __4. 텍스트 수치화 모델을 사용하여 문서에서 키워드 추출__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서에서 키워드를 추출합니다.

In [11]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT *
FROM nsmc_test
LIMIT 10 OFFSET 40

,id,document,label,convert_result,keyword
0,7928591,"일단 엠씨 역량 부족이 가장 큰 문제인듯. 시간 없다고, 재미없다고 게스트 말 끊는...",0,"[0.0035488934, -0.01865615, 0.043746103, -0.01...","{'keyword': ['라고 끝 엠씨가노답임', '방 왜 ᆷ', '재미없 게스트 ..."
1,8380896,방금 슬쩍 봤는데 '인삼' 처럼 생긴 '산삼' 암수가 서로 토닥이다가 주인공에게 잡...,0,"[-0.036318187, -0.0037342785, 0.009260191, -0....","{'keyword': ['산삼 암수 서로', '는데 인삼 처럼', '방금 슬쩍 는데..."
2,8914833,신세계에 비교하는 거 진심 빡친다. 신성모독,1,"[0.003922502, -0.003720448, 0.024366783, 0.094...","{'keyword': ['빡 치 신성모독', '비교 거 진심', '신성모독', '신..."
3,9702804,킬링타임 최고다 열자채워,1,"[0.049014356, -0.023481527, -0.043345053, 0.03...","{'keyword': ['킬링타임 최고', '킬링타임 최고 다', '킬링타임', '..."
4,10097178,명불허전 선동영화 10자,0,"[0.016989384, 0.014749343, 0.028998429, -0.020...","{'keyword': ['명불허전 선동 영화', '명불허전 선동', '명불허전', ..."
5,9245590,결말이 마음에 든다. 소설을 먼저 읽었던 터라 결말이 마음에 든다. 세상의 모든 정...,1,"[-0.0068695513, -0.031204086, 0.029428925, 0.0...","{'keyword': ['결말 마음 들', '정혜 힘내 어요', '소설 먼저 읽',..."
6,4227156,기대했는데.. 시나리오는 ...어떻게 결말이 그렇지?..요즘 막장 드라마 영향인가..,0,"[0.006298935, -0.032475553, 0.074941054, 0.025...","{'keyword': ['막장 드라마 영향', '결말 그렇 지', '데 시나리오 어..."
7,7313784,인기연예인 몇명 출현한다고 재미있는 영화냐 잡것들아긴급조치19호가 그렇게 만들고 망...,0,"[0.040298615, 0.0075149518, 0.0776514, -0.0007...","{'keyword': ['영화 냐 잡것', '인기 연예인 몇', '출현 ᆫ다고 재미..."
8,3582981,좋았음!!,1,"[0.06824996, 0.013203928, -0.06663332, 0.03160...","{'keyword': ['좋 음', '좋', '음'], 'score': [0.546..."
9,9906269,멜로와 스릴러를 이렇게 조합할수 있다니 ...,1,"[-0.00065220613, -0.03436944, 0.04084344, 0.02...","{'keyword': ['멜로 스릴러', '스릴러 이렇 조합', '멜로', '조합'..."


In [12]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        use_stopwords=True
        )
    AS
    SELECT *
    FROM nsmc_test
    LIMIT 10
)

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[영화 네요, 네요, 번 영화, 꼭 번, 영화]","[0.5873, 0.4716, 0.3792, 0.3737, 0.3444]"
1,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[조연 ~여주, 안드로메다 ~럽, 권선징악 안드로메다, 작가 빙의, 나쁘 여자]","[0.5012, 0.4497, 0.4431, 0.334, 0.3054]"
2,유쾌하고 신나는 스피드 코미디.,1,"[스피드 코미디, 유쾌, 코미디, 스피드, 신 나]","[0.7314, 0.6205, 0.5518, 0.4755, 0.3516]"
3,정말 따뜻했던 드라마ㅠㅠ,1,"[따뜻하 드라마, 정말 따뜻하, 정말, 따뜻하, 드라마]","[0.7332, 0.7013, 0.5861, 0.5732, 0.4854]"
4,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[연출력 그냥, 하지만, 가능 뻔, 똥 차라리, 살인마 주인공]","[0.407, 0.2647, 0.2608, 0.2479, 0.2038]"
5,느와르에 멜로에 신파극까지······. 짬뽕이 아니라 꿀꿀이죽,0,"[신파극 짬뽕, 느와르, 꿀꿀이죽, 짬뽕, 멜로]","[0.4345, 0.3525, 0.3325, 0.3175, 0.2111]"
6,다시 보고 싶네요~,1,"[다시 싶, 싶 네요, 다시, 네요, 싶]","[0.7056, 0.6488, 0.617, 0.5583, 0.4922]"
7,짱이야... 진짜 극장을 나오는데 멋진 소설을 한권 읽은 듯한 기분을 느낌. 재미는...,1,"[물론 감동, 진짜 극장, 짱 야, 영화 꼭, 철철 넘치]","[0.4624, 0.406, 0.3751, 0.3538, 0.3002]"
8,유일하게 아들과 같이볼수있는 만화영화,1,"[만화 영화, 있 만화, 아들 같이, 유일, 있]","[0.6316, 0.551, 0.4587, 0.3178, 0.2874]"
9,남자라면 봐라 꼭봐라 두번봐라 세번봐라 계속봐라,1,"[세 번, 계속 어라, 보 어라, 꼭, 남자 라면]","[0.4276, 0.4044, 0.3484, 0.3215, 0.2128]"


In [13]:
%%thanosql
SELECT * FROM (
    SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, json_array_elements(keyword -> 'score') AS score
        FROM (
            SEARCH KEYWORD 
            USING nsmc_text_search_model
            OPTIONS (
                text_col='document',
                use_stopwords=True
                )
            AS
            SELECT *
            FROM nsmc_test
            LIMIT 10
        )
    )
WHERE score > 0.5

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,영화 네요,0.5873
1,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,조연 ~여주,0.5012
2,유쾌하고 신나는 스피드 코미디.,1,스피드 코미디,0.7314
3,유쾌하고 신나는 스피드 코미디.,1,유쾌,0.6205
4,유쾌하고 신나는 스피드 코미디.,1,코미디,0.5518
5,정말 따뜻했던 드라마ㅠㅠ,1,따뜻하 드라마,0.7332
6,정말 따뜻했던 드라마ㅠㅠ,1,정말 따뜻하,0.7013
7,정말 따뜻했던 드라마ㅠㅠ,1,정말,0.5861
8,정말 따뜻했던 드라마ㅠㅠ,1,따뜻하,0.5732
9,다시 보고 싶네요~,1,다시 싶,0.7056


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
            <ul>
                <li>"lang": 사용할 언어 (str, optional, 'ko'|'en', default: 'ko')</li>
                <li>"text_col": 텍스트가 있는 열 이름 (str, default: 'text')</li>
                <li>"ngram_range": 키워드의 최소 단어 수와 최대 단어수 예) [1, 3]. 대부분의 상황에서 키워드는 최대 단어 수에 맞춰 추출됨 (list[int, int], optional, default: [1, 2])</li>
                <li>"top_n": 추출할 키워드의 수, 유사도가 높은 순서대로 (int, optional, default: 5)</li>
                <li>"diversity": 추출될 키워드의 다양성. 높을 수록 기존에 추출된 키워드와 유사한 키워드는 다시 추출되지 않음 0 <= diversity <= 1 (float, optional, default: 0.5)</li>
                <li>"use_stopwords": 큰 의미가 없는 단어(불용어)를 제외할 지 여부 (bool, optional, True|False, default: True)</li>
                <li>"threshold": 추출할 키워드의 유사도 수치의 최소값 (float, optional, default: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __5. 두 방법 결합하여 사용__

In [ ]:
            SEARCH KEYWORD 
            USING nsmc_text_search_model
            OPTIONS (
                text_col='document',
                use_stopwords=True
                )
            AS
            SELECT *
            FROM nsmc_test
            LIMIT 10
        )

In [14]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT document, label, score
    FROM (
        SEARCH TEXT
        USING nsmc_text_search_model
        OPTIONS (
            search_by='text',
            search_input='가볍게 볼 수 있는 코미디 영화',
            emb_col='convert_result',
            result_col='score'
            )
        AS
        SELECT *
        FROM nsmc_test
        )
    ORDER BY score DESC
    LIMIT 10
)

,document,label,score,keyword
0,순수하게 보면 참으로 재밌는 영화,1,0.770930,"{'keyword': ['참으로 재밌 영화', '영화', '참으로', '순수', '..."
1,코미디로도 사람의 마음을 울릴수 있느 영화,1,0.728467,"{'keyword': ['울리 있느 영화', '마음 울리 있느', '코미디 마음 울..."
2,감동적인 영화,1,0.709423,"{'keyword': ['감동 영화', '감동', '영화'], 'score': [0..."
3,보고나면 기분 좋은 영화,1,0.700379,"{'keyword': ['기분 좋 영화', '기분 좋', '좋 영화', '나 기분'..."
4,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,0.673917,"{'keyword': ['보 만 영화', '모르 잼 보', '잼', '시간 가', ..."
5,긴박한 영화만 좋아한다면 보지말구 영화 자체를 즐기는 사람이라면 감탄할 영화,1,0.657594,"{'keyword': ['영화 좋아하 ᆫ다면', '영화 자체 즐기', '라면 감탄 ..."
6,어릴 때 보고 충격먹은 영화,0,0.656170,"{'keyword': ['충격 먹 영화', '영화', '어리 충격 먹', '먹', ..."
7,정말 자꾸 보게되는 드라마,1,0.656073,"{'keyword': ['정말 자꾸 드라마', '자꾸 드라마', '정말 자꾸', '..."
8,다시보고 싶은영화로 추천,1,0.649943,"{'keyword': ['다시 싶 영화', '싶 영화 추천', '다시', '영화',..."
9,보는 내내 정말 열받았던 영화,1,0.644611,"{'keyword': ['열 받 영화', '내내 정말', '받 영화', '정말 열 ..."


In [15]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT document, label, score
        FROM (
            SEARCH TEXT
            USING nsmc_text_search_model
            OPTIONS (
                search_by='text',
                search_input='가볍게 볼 수 있는 코미디 영화',
                emb_col='convert_result',
                result_col='score'
                )
            AS
            SELECT *
            FROM nsmc_test
            )
        ORDER BY score DESC
        LIMIT 10
    )
)

,document,label,keywords,score
0,순수하게 보면 참으로 재밌는 영화,1,"[참으로 재밌 영화, 영화, 참으로, 순수, 보 참으로]","[0.7977, 0.5444, 0.488, 0.4425, 0.4196]"
1,코미디로도 사람의 마음을 울릴수 있느 영화,1,"[울리 있느 영화, 마음 울리 있느, 코미디 마음 울리, 영화, 코미디]","[0.7074, 0.6562, 0.6455, 0.4326, 0.3396]"
2,감동적인 영화,1,"[감동 영화, 감동, 영화]","[0.9011, 0.7226, 0.6753]"
3,보고나면 기분 좋은 영화,1,"[기분 좋 영화, 기분 좋, 좋 영화, 나 기분, 영화]","[0.8507, 0.6963, 0.6871, 0.5516, 0.5449]"
4,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,"[보 만 영화, 모르 잼 보, 잼, 시간 가, 줄 모르]","[0.4863, 0.463, 0.3652, 0.3419, 0.2849]"
5,긴박한 영화만 좋아한다면 보지말구 영화 자체를 즐기는 사람이라면 감탄할 영화,1,"[영화 좋아하 ᆫ다면, 영화 자체 즐기, 라면 감탄 영화, 긴박 영화, 자체 즐기 라면]","[0.6724, 0.6183, 0.503, 0.411, 0.3719]"
6,어릴 때 보고 충격먹은 영화,0,"[충격 먹 영화, 영화, 어리 충격 먹, 먹, 어리]","[0.6936, 0.5765, 0.4529, 0.3381, 0.3288]"
7,정말 자꾸 보게되는 드라마,1,"[정말 자꾸 드라마, 자꾸 드라마, 정말 자꾸, 정말, 드라마]","[0.8294, 0.6935, 0.6859, 0.5779, 0.4957]"
8,다시보고 싶은영화로 추천,1,"[다시 싶 영화, 싶 영화 추천, 다시, 영화, 추천]","[0.7807, 0.7311, 0.5336, 0.5184, 0.4005]"
9,보는 내내 정말 열받았던 영화,1,"[열 받 영화, 내내 정말, 받 영화, 정말 열 받, 영화]","[0.6225, 0.6153, 0.5458, 0.5418, 0.5415]"


In [16]:
%%thanosql
SELECT *
FROM
    (SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, json_array_elements(keyword -> 'score') AS score
    FROM (
        SEARCH KEYWORD
        USING nsmc_text_search_model
        OPTIONS (
            text_col='document',
            ngram_range=[1, 3],
            use_stopwords=True
            )
        AS (
            SELECT document, label, score
            FROM (
                SEARCH TEXT
                USING nsmc_text_search_model
                OPTIONS (
                    search_by='text',
                    search_input='최고의 액션 영화',
                    emb_col='convert_result',
                    result_col='score'
                    )
                AS
                SELECT *
                FROM nsmc_test
                WHERE document LIKE '%%판타지%%'
                )
            ORDER BY score DESC
            LIMIT 10
        )
    )
)
WHERE score > 0.3

,document,label,keywords,score
0,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,바로 판타지 모험,0.5894
1,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,이런 바로 판타지,0.5480
2,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,정말 재밌 이런,0.5339
3,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,영화 현실,0.4994
4,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,이런 바로,0.3782
5,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,퓨전 판타지 라니,0.5999
6,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,그냥 찬양 ᆷ,0.5433
7,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,라니 그냥,0.4592
8,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,찬양,0.3601
9,1700만 1700만 해되서 봤다가 욕만하며 참고 참으며 보다 결국 사분의 일은 스...,0,삼류 판타지 아류작,0.4006


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __6. 튜토리얼을 마치며__

이번 튜토리얼에서는 nsmc 영화 리뷰 데이터 세트를 사용하여 텍스트 수치화와 수치화 결과를 바탕으로한 유사 텍스트 검색, 키워드 추출을 진행해 보았습니다. 이번 튜토리얼에서는 모델의 수치화 성능보다는 작동 위주의 설명으로 진행하였습니다. 나만의 텍스트 수치화 모델을 만들어 다양한 형태의 비정형 데이터 세트에 검색 기능을 추가하고 Auto-ML 기법을 이용한 나만의 모델을 배포할 수 있습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>유사 텍스트 검색 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>